In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setup driver
driver = webdriver.Chrome()
driver.set_window_size(1300, 900)

# Link halaman Tokopedia
tokped_link = "https://www.tokopedia.com/search?st=&q=flat%20shoes&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource="
driver.get(tokped_link)

data_produk = []

for page in range(3):  # Lakukan scraping pada 3 halaman
    time.sleep(3)

    # Scroll untuk memastikan semua produk di halaman terload
    for i in range(14):
        driver.execute_script(f"window.scrollTo(0, {500 * i});")
        time.sleep(1)

    # Parsing halaman setelah scroll
    data = BeautifulSoup(driver.page_source, 'html.parser')

    # Ambil data dari elemen produk
    for area in data.find_all('div', class_='css-5wh65g'):
        try:
            link = area.find('a')['href']
            data_produk.append(link)  # Menyimpan hanya link produk
        except Exception as e:
            print("Error retrieving product link:", e)

    # Coba klik tombol 'Laman berikutnya' dengan explicit wait
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label='Laman berikutnya']"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", next_button)
        time.sleep(2)  # Tambahkan sedikit delay untuk memastikan elemen sudah stabil
        next_button.click()
        time.sleep(2)  # Tunggu halaman berikutnya untuk dimuat
    except Exception as e:
        print("Tidak dapat mengklik tombol 'Laman berikutnya':", e)
        break

# Simpan data ke file CSV
df = pd.DataFrame(data_produk)
df.to_csv('result_crawling_tokped_with_selenium.csv', index=False, header=False)
print('Link produk telah tersimpan ke result_crawling_tokped_with_selenium.csv')

# Tutup driver
driver.quit()

Link produk telah tersimpan ke result_crawling_tokped_with_selenium.csv
